In [5]:
import pandas as pd
import numpy as np
from cmath import pi
import random
import sys
from zero_point import zpt
zpt.load_tables()

input data

In [8]:
id = 0
data = pd.read_csv(r"gaia_dr3_split/GaiaSource_DR3_%s.csv" %(id),header = 0, usecols=['b', 'bp_rp','designation','phot_g_mean_mag','nu_eff_used_in_astrometry', 'pseudocolour', 'ecl_lat', 'astrometric_params_solved', \
    'ra','dec','parallax','pmra','pmdec','radial_velocity','ra_error','dec_error','parallax_error','pmra_error','pmdec_error','radial_velocity_error',\
        'ra_dec_corr','ra_parallax_corr','ra_pmra_corr','ra_pmdec_corr','dec_parallax_corr','dec_pmra_corr','dec_pmdec_corr','parallax_pmra_corr','parallax_pmdec_corr','pmra_pmdec_corr'])
#d1 = len(data)
data = data[data['parallax']>0]
#d2 = len(data)
#print('negative plx number', d1-d2)


Origin data bias correction

In [9]:
############################
#  Reference: Lindegren et al. 2021
############################

zpvals = data.apply(zpt.zpt_wrapper,axis=1) 
zpvals = np.array(zpvals)

In [10]:
############################
#  Reference: Ding, Y., Liao, S., Wu, Q., Qi, Z., & Tang, Z. 2024, A&A, 691, A81
############################

from numpy import deg2rad, cos,sin,array, diagonal

def calc_pzpo(phot_g_mean_mag, colour, beta, filename):
    '''
    calculate parallax bias for five-parameter solutions located in |b|<=20 deg in Gaia DR3.
    
    Input parameter: 
        phot_g_mean_mag: G magnitude
        colour: effective wavenumber 
        beta: eclipic latitude 
        b: galactic latitude
        filename: path to the file with the coefficients
    
    Output parameter:
        parallax bias in microarcsecond
    '''
    sinBeta = np.sin(beta)
    # reads the file (.txt)
    input_file = np.genfromtxt(filename, delimiter=',')

    # auxiliary variables j and k
    j = list(map(int, input_file[0, 1:]))
    k = list(map(int, input_file[1, 1:]))
    # g vector
    g = input_file[2:, 0]
    # coefficients
    q_jk = input_file[2:, 1:]
    # shape
    n, m = q_jk.shape
    
    
    # basis functions evaluated at colour and ecl_lat
    c = [np.ones_like(colour),
         np.max((-0.24 * np.ones_like(colour), np.min((0.24 * np.ones_like(colour), colour - 1.48), axis=0)), axis=0),
         np.min((0.24 * np.ones_like(colour), np.max((np.zeros_like(colour), 1.48 - colour), axis=0)), axis=0) ** 3,
         np.min((np.zeros_like(colour), colour - 1.24), axis=0),
         np.max((np.zeros_like(colour), colour - 1.72), axis=0)]
    b = [np.ones_like(sinBeta), sinBeta, sinBeta ** 2 - 1. / 3]

    # coefficients must be interpolated between g(left) and g(left+1)
    # find the bin in g where gMag is
    ig = np.max((np.zeros_like(phot_g_mean_mag),
                 np.min((np.ones_like(phot_g_mean_mag) * (n - 2), np.digitize(phot_g_mean_mag, g, right=False) - 1),
                        axis=0)), axis=0).astype(int)

    # interpolate coefficients to gMag:
    h = np.max((np.zeros_like(phot_g_mean_mag),
                np.min((np.ones_like(phot_g_mean_mag), (phot_g_mean_mag - g[ig]) / (g[ig + 1] - g[ig])), axis=0)),
               axis=0)

    # sum over the product of the coefficients to get the zero-point
    pzpo = np.sum([((1 - h) * q_jk[ig, i] + h * q_jk[ig + 1, i]) * c[j[i]] * b[k[i]] for i in range(m)], axis=0)
    
    return pzpo

G = np.array(data['phot_g_mean_mag'])
color = np.array(data['nu_eff_used_in_astrometry']) # effective wavenumber 
beta = np.array(data['ecl_lat'])
N = np.array(data['astrometric_params_solved'])  # 5-para is N=31
b = np.array(data['b'])
path = "Parallax-bias-correction-in-the-Galactic-plane-main/coefficients.txt"
plx = []
for i in range(len(zpvals)):
    if N[i] == 31 and np.abs(b[i])<=2:
        pzpo = 0.001* calc_pzpo(G[i], color[i], beta[i], path) # parallax bias in mas
        plxi = np.array(data['parallax'])[i]-pzpo
    else: 
        if np.isnan(zpvals[i]) == True:
            plxi = np.array(data['parallax'])[i]
        else:
            plxi = np.array(data['parallax'])[i]-zpvals[i]
    plx.append(plxi)


data['parallax'] = np.array(plx)
plx = np.array(data['parallax'])   #mas
plx_e = np.array(data['parallax_error'])   #mas
data['plx_e/plx']=plx_e/plx
data = data[(data['plx_e/plx']<0.2) & (data['plx_e/plx']>0)]  #select fractional parallax uncertainty < 20 %


In [12]:
############################
#  Reference: Cantat-Gaudin, Tristan, & Brandt, Timothy D. 2021, AA, 649, A124
############################
def edr3ToICRF (pmra ,pmdec ,ra ,dec ,G):
    """
    Input: source position , coordinates ,
    and G magnitude from Gaia EDR3.
    Output: corrected proper motion.
    """
    if G >=13:
        return pmra , pmdec

    def sind(x):
        return np.sin(np.radians(x))
    def cosd(x):
        return np.cos(np.radians(x))

    table1=""" 0.0 9.0 18.4 33.8 -11.3
    9.0 9.5 14.0 30.7 -19.4
    9.5 10.0 12.8 31.4 -11.8
    10.0 10.5 13.6 35.7 -10.5
    10.5 11.0 16.2 50.0 2.1
    11.0 11.5 19.4 59.9 0.2
    11.5 11.75 21.8 64.2 1.0
    11.75 12.0 17.7 65.6 -1.9
    12.0 12.25 21.3 74.8 2.1
    12.25 12.5 25.7 73.6 1.0
    12.5 12.75 27.3 76.6 0.5
    12.75 13.0 34.9 68.9 -2.9 """
    table1 = np. fromstring (table1 ,sep=' ').reshape ((12 ,5)).T
    Gmin = table1[0]
    Gmax = table1[1]
    #pick the appropriate omegaXYZ for the source’s magnitude:
    omegaX = table1[2][( Gmin <=G)&(Gmax >G)][0]
    omegaY = table1[3][( Gmin <=G)&(Gmax >G)][0]
    omegaZ = table1[4][( Gmin <=G)&(Gmax >G)][0]
    pmraCorr = -1* sind(dec)*cosd(ra)*omegaX -sind(dec)*sind(ra)*omegaY + cosd(dec)*omegaZ
    pmdecCorr = sind(ra)*omegaX -cosd(ra)*omegaY
    return pmra -pmraCorr /1000. , pmdec - pmdecCorr /1000.

ra = np.array(data['ra'])   #deg
dec = np.array(data['dec'])   #deg
G = np.array(data['phot_g_mean_mag'])
pmra_o = np.array(data['pmra'])   #mas/yr
pmdec_o = np.array(data['pmdec'])   #mas/yr
pmra=[]
pmdec=[]
for i in range(len(ra)):
    if np.isnan(G[i]) == True:  
        a = pmra_o[i]
        d = pmdec_o[i]
    else:
        a,d = edr3ToICRF (pmra_o[i] ,pmdec_o[i] ,ra[i] ,dec[i] ,G[i])
    pmra.append(a)
    pmdec.append(d)
data['pmra'] = np.array(pmra)
data['pmdec']= np.array(pmdec)
data.to_csv(r"GDR3-correct-icrs/GaiaSource_DR3_%s.csv" %(id), index=False, sep=',')

Linear error propagation in Equatorial coordinate system

In [15]:
data = pd.read_csv(r"GDR3-correct-icrs/GaiaSource_DR3_%s.csv" %(id),header = 0, usecols=['bp_rp','designation','phot_g_mean_mag','nu_eff_used_in_astrometry', 'pseudocolour', 'ecl_lat', 'astrometric_params_solved', \
    'ra','dec','parallax','pmra','pmdec','radial_velocity','ra_error','dec_error','parallax_error','pmra_error','pmdec_error','radial_velocity_error',\
        'ra_dec_corr','ra_parallax_corr','ra_pmra_corr','ra_pmdec_corr','dec_parallax_corr','dec_pmra_corr','dec_pmdec_corr','parallax_pmra_corr','parallax_pmdec_corr','pmra_pmdec_corr'])

designation = np.array(data['designation'])
bp_rp = np.array(data['bp_rp'])
ra = np.array(data['ra'])   #deg
dec = np.array(data['dec'])   #deg
plx = np.array(data['parallax'])   #mas
pmra = np.array(data['pmra'])   #mas/yr
pmdec = np.array(data['pmdec'])   #mas/yr
rv = np.array(data['radial_velocity'])   #km/s

ras_e = np.array(data['ra_error'])   #mas  ra*_error, used in MC method
ra_e = ras_e/cos(dec*pi/180)   # ra_error 
dec_e = np.array(data['dec_error'])   #mas
plx_e = np.array(data['parallax_error'])   #mas
pmra_e = np.array(data['pmra_error'])   #mas/yr
pmdec_e = np.array(data['pmdec_error'])   #mas/yr
rv_e = np.array(data['radial_velocity_error'])   #km/s

ra_dec_corr = np.array(data['ra_dec_corr'])
ra_plx_corr = np.array(data['ra_parallax_corr'])
ra_pmra_corr = np.array(data['ra_pmra_corr'])
ra_pmdec_corr = np.array(data['ra_pmdec_corr'])
dec_plx_corr = np.array(data['dec_parallax_corr'])
dec_pmra_corr = np.array(data['dec_pmra_corr'])
dec_pmdec_corr = np.array(data['dec_pmdec_corr'])
plx_pmra_corr = np.array(data['parallax_pmra_corr'])
plx_pmdec_corr = np.array(data['parallax_pmdec_corr'])
pmra_pmdec_corr = np.array(data['pmra_pmdec_corr'])


In [22]:
# celestial -> Cartesian
# km/s->AU/yr = 4.740470463533348

from numpy import deg2rad, cos,sin,array, diagonal
def ct(ra,dec,plx,pmra,pmdec,rv):
    alpha=deg2rad(ra)   #rad
    delta=deg2rad(dec)  #rad
    r=1000/plx   #pc
    x=r*cos(delta)*cos(alpha)   #pc
    y=r*cos(delta)*sin(alpha)   #pc
    z=r*sin(delta)   #pc
    vx=rv*cos(delta)*cos(alpha)+(-r*pmra*sin(alpha)/1000-r*pmdec*sin(delta)*cos(alpha)/1000)*4.740470463533348   #km/s
    vy=rv*cos(delta)*sin(alpha)+(r*pmra*cos(alpha)/1000-r*pmdec*sin(delta)*sin(alpha)/1000)*4.740470463533348   #km/s
    vz=rv*sin(delta)+r*pmdec*cos(delta)/1000*4.740470463533348   #km/s
    return array([x,y,z,vx,vy,vz])


In [23]:
# Propagation verification
import astropy.units as u
from astropy.coordinates import SkyCoord

ra0,dec0,plx0,pmra0,pmdec0,rv0 = ra[0],dec[0],plx[0],pmra[0],pmdec[0],rv[0]
x,y,z,vx,vy,vz = ct(ra0,dec0,plx0,pmra0,pmdec0,rv0)
c = SkyCoord(ra=ra0*u.deg, dec=dec0*u.deg, distance=1/plx0*u.kpc, pm_ra_cosdec=pmra0*u.mas/u.yr, pm_dec=pmdec0*u.mas/u.yr, radial_velocity=rv0*u.km/u.s, frame='icrs')
x1 = c.cartesian.x.to(u.pc)
y1 = c.cartesian.y.to(u.pc)
z1 = c.cartesian.z.to(u.pc)
vx1 = c.velocity.d_x.to(u.km/u.s)
vy1 = c.velocity.d_y.to(u.km/u.s)
vz1 = c.velocity.d_z.to(u.km/u.s)
print(c.velocity.d_z.to(u.km/u.s)/c.velocity.d_z.to(u.AU/u.yr))  #km/s -> AU/yr

print(x,y,z,vx,vy,vz)
print(x1,y1,z1,vx1,vy1,vz1)

4.740470463533348 km yr / (AU s)
15.981370769227338 -90.40130996492876 -63.33760508199503 12.349039956574721 32.92978536057082 -15.54929144012951
15.98137076922734 pc -90.40130996492876 pc -63.33760508199503 pc 12.349039956574723 km / s 32.92978536057082 km / s -15.549291440129506 km / s


In [24]:
# Variance-Covariance matrix
def cov_obs(ra_e,dec_e,plx_e, pmra_e, pmde_e, rv_e,ra_dec_corr, ra_plx_corr, ra_pmra_corr, ra_pmdec_corr,ra_rv_corr,dec_plx_corr, dec_pmra_corr, dec_pmdec_corr,dec_rv_corr,plx_pmra_corr, plx_pmdec_corr,plx_rv_corr,pmra_pmdec_corr,pmra_rv_corr,pmdec_rv_corr):
    e = (ra_e, dec_e, plx_e, pmra_e, pmde_e, rv_e )   #error matrix
    k = np.matrix([[0,ra_dec_corr, ra_plx_corr, ra_pmra_corr, ra_pmdec_corr, ra_rv_corr],\
         [0, 0, dec_plx_corr, dec_pmra_corr, dec_pmdec_corr, dec_rv_corr],\
              [0, 0, 0, plx_pmra_corr, plx_pmdec_corr, plx_rv_corr],\
                  [0, 0, 0, 0, pmra_pmdec_corr,pmra_rv_corr],\
                      [0,0,0,0,0,pmdec_rv_corr],[0,0,0,0,0,0]])
    corr_i = np.identity(6) + k + k.T   #correlation matrix
    err = np.identity(6) * e
    cov_obs = err @ corr_i @ err
    return cov_obs 

#Correlation coefficient matrix
def corr(ra_e, dec_e, plx_e, pmra_e, pmdec_e, rv_e,VC):  
    e = np.matrix([ra_e, dec_e, plx_e, pmra_e, pmdec_e, rv_e] )   #error matrix
    ee = e.T @ e
    cor = VC/ee
    return cor 


In [25]:
# Jacobian matrix

def J_car(ra_i,dec_i,parallax_i,pmras_i,pmdec_i,rv_i):
    dlt_i = dec_i*pi/180
    alp_i = ra_i*pi/180
    dx = np.array([-1000*sin(alp_i)*cos(dlt_i)/parallax_i, -1000*sin(dlt_i)*cos(alp_i)/parallax_i, -1000*cos(alp_i)*cos(dlt_i)/parallax_i**2, 0, 0, 0])
    dy = np.array([1000*cos(alp_i)*cos(dlt_i)/parallax_i, -1000*sin(alp_i)*sin(dlt_i)/parallax_i, -1000*sin(alp_i)*cos(dlt_i)/parallax_i**2, 0, 0, 0])
    dz = np.array([0, 1000*cos(dlt_i)/parallax_i, -1000*sin(dlt_i)/parallax_i**2, 0, 0, 0])
    dvx = np.array([-rv_i*sin(alp_i)*cos(dlt_i) + 4.74047046353335*pmdec_i*sin(alp_i)*sin(dlt_i)/parallax_i - 4.74047046353335*pmras_i*cos(alp_i)/parallax_i, -rv_i*sin(dlt_i)*cos(alp_i) - 4.74047046353335*pmdec_i*cos(alp_i)*cos(dlt_i)/parallax_i, 4.74047046353335*pmdec_i*sin(dlt_i)*cos(alp_i)/parallax_i**2 + 4.74047046353335*pmras_i*sin(alp_i)/parallax_i**2, -4.74047046353335*sin(alp_i)/parallax_i, -4.74047046353335*sin(dlt_i)*cos(alp_i)/parallax_i, cos(alp_i)*cos(dlt_i)])
    dvy = np.array([rv_i*cos(alp_i)*cos(dlt_i) - 4.74047046353335*pmdec_i*sin(dlt_i)*cos(alp_i)/parallax_i - 4.74047046353335*pmras_i*sin(alp_i)/parallax_i, -rv_i*sin(alp_i)*sin(dlt_i) - 4.74047046353335*pmdec_i*sin(alp_i)*cos(dlt_i)/parallax_i, 4.74047046353335*pmdec_i*sin(alp_i)*sin(dlt_i)/parallax_i**2 - 4.74047046353335*pmras_i*cos(alp_i)/parallax_i**2, 4.74047046353335*cos(alp_i)/parallax_i, -4.74047046353335*sin(alp_i)*sin(dlt_i)/parallax_i, sin(alp_i)*cos(dlt_i)])
    dvz = np.array([0, rv_i*cos(dlt_i) - 4.74047046353335*pmdec_i*sin(dlt_i)/parallax_i, -4.74047046353335*pmdec_i*cos(dlt_i)/parallax_i**2, 0, 4.74047046353335*cos(dlt_i)/parallax_i, sin(dlt_i)])
    return np.array([dx,dy,dz,dvx,dvy,dvz])

In [26]:
# Hessian matrix

def H(ra_i,dec_i,parallax_i,pmras_i,pmdec_i,rv_i):
    alp_i = ra_i*pi/180 
    dlt_i = dec_i*pi/180
    xHi = [[-1000*cos(alp_i)*cos(dlt_i)/parallax_i, 1000*sin(alp_i)*sin(dlt_i)/parallax_i, 1000*sin(alp_i)*cos(dlt_i)/parallax_i**2, 0, 0, 0], [1000*sin(alp_i)*sin(dlt_i)/parallax_i, -1000*cos(alp_i)*cos(dlt_i)/parallax_i, 1000*sin(dlt_i)*cos(alp_i)/parallax_i**2, 0, 0, 0], [1000*sin(alp_i)*cos(dlt_i)/parallax_i**2, 1000*sin(dlt_i)*cos(alp_i)/parallax_i**2, 2000*cos(alp_i)*cos(dlt_i)/parallax_i**3, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0]]
    yHi = [[-1000*sin(alp_i)*cos(dlt_i)/parallax_i, -1000*sin(dlt_i)*cos(alp_i)/parallax_i, -1000*cos(alp_i)*cos(dlt_i)/parallax_i**2, 0, 0, 0], [-1000*sin(dlt_i)*cos(alp_i)/parallax_i, -1000*sin(alp_i)*cos(dlt_i)/parallax_i, 1000*sin(alp_i)*sin(dlt_i)/parallax_i**2, 0, 0, 0], [-1000*cos(alp_i)*cos(dlt_i)/parallax_i**2, 1000*sin(alp_i)*sin(dlt_i)/parallax_i**2, 2000*sin(alp_i)*cos(dlt_i)/parallax_i**3, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0]]
    zHi = [[0, 0, 0, 0, 0, 0], [0, -1000*sin(dlt_i)/parallax_i, -1000*cos(dlt_i)/parallax_i**2, 0, 0, 0], [0, -1000*cos(dlt_i)/parallax_i**2, 2000*sin(dlt_i)/parallax_i**3, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0]]
    vxHi = [[-rv_i*cos(alp_i)*cos(dlt_i) + 4.74047046353335*pmdec_i*sin(dlt_i)*cos(alp_i)/parallax_i + 4.74047046353335*pmras_i*sin(alp_i)/parallax_i, rv_i*sin(alp_i)*sin(dlt_i) + 4.74047046353335*pmdec_i*sin(alp_i)*cos(dlt_i)/parallax_i, -4.74047046353335*pmdec_i*sin(alp_i)*sin(dlt_i)/parallax_i**2 + 4.74047046353335*pmras_i*cos(alp_i)/parallax_i**2, -4.74047046353335*cos(alp_i)/parallax_i, 4.74047046353335*sin(alp_i)*sin(dlt_i)/parallax_i, -sin(alp_i)*cos(dlt_i)], [rv_i*sin(alp_i)*sin(dlt_i) + 4.74047046353335*pmdec_i*sin(alp_i)*cos(dlt_i)/parallax_i, -rv_i*cos(alp_i)*cos(dlt_i) + 4.74047046353335*pmdec_i*sin(dlt_i)*cos(alp_i)/parallax_i, 4.74047046353335*pmdec_i*cos(alp_i)*cos(dlt_i)/parallax_i**2, 0, -4.74047046353335*cos(alp_i)*cos(dlt_i)/parallax_i, -sin(dlt_i)*cos(alp_i)], [-4.74047046353335*pmdec_i*sin(alp_i)*sin(dlt_i)/parallax_i**2 + 4.74047046353335*pmras_i*cos(alp_i)/parallax_i**2, 4.74047046353335*pmdec_i*cos(alp_i)*cos(dlt_i)/parallax_i**2, -9.4809409270667*pmdec_i*sin(dlt_i)*cos(alp_i)/parallax_i**3 - 9.4809409270667*pmras_i*sin(alp_i)/parallax_i**3, 4.74047046353335*sin(alp_i)/parallax_i**2, 4.74047046353335*sin(dlt_i)*cos(alp_i)/parallax_i**2, 0], [-4.74047046353335*cos(alp_i)/parallax_i, 0, 4.74047046353335*sin(alp_i)/parallax_i**2, 0, 0, 0], [4.74047046353335*sin(alp_i)*sin(dlt_i)/parallax_i, -4.74047046353335*cos(alp_i)*cos(dlt_i)/parallax_i, 4.74047046353335*sin(dlt_i)*cos(alp_i)/parallax_i**2, 0, 0, 0], [-sin(alp_i)*cos(dlt_i), -sin(dlt_i)*cos(alp_i), 0, 0, 0, 0]]
    vyHi = [[-rv_i*sin(alp_i)*cos(dlt_i) + 4.74047046353335*pmdec_i*sin(alp_i)*sin(dlt_i)/parallax_i - 4.74047046353335*pmras_i*cos(alp_i)/parallax_i, -rv_i*sin(dlt_i)*cos(alp_i) - 4.74047046353335*pmdec_i*cos(alp_i)*cos(dlt_i)/parallax_i, 4.74047046353335*pmdec_i*sin(dlt_i)*cos(alp_i)/parallax_i**2 + 4.74047046353335*pmras_i*sin(alp_i)/parallax_i**2, -4.74047046353335*sin(alp_i)/parallax_i, -4.74047046353335*sin(dlt_i)*cos(alp_i)/parallax_i, cos(alp_i)*cos(dlt_i)], [-rv_i*sin(dlt_i)*cos(alp_i) - 4.74047046353335*pmdec_i*cos(alp_i)*cos(dlt_i)/parallax_i, -rv_i*sin(alp_i)*cos(dlt_i) + 4.74047046353335*pmdec_i*sin(alp_i)*sin(dlt_i)/parallax_i, 4.74047046353335*pmdec_i*sin(alp_i)*cos(dlt_i)/parallax_i**2, 0, -4.74047046353335*sin(alp_i)*cos(dlt_i)/parallax_i, -sin(alp_i)*sin(dlt_i)], [4.74047046353335*pmdec_i*sin(dlt_i)*cos(alp_i)/parallax_i**2 + 4.74047046353335*pmras_i*sin(alp_i)/parallax_i**2, 4.74047046353335*pmdec_i*sin(alp_i)*cos(dlt_i)/parallax_i**2, -9.4809409270667*pmdec_i*sin(alp_i)*sin(dlt_i)/parallax_i**3 + 9.4809409270667*pmras_i*cos(alp_i)/parallax_i**3, -4.74047046353335*cos(alp_i)/parallax_i**2, 4.74047046353335*sin(alp_i)*sin(dlt_i)/parallax_i**2, 0], [-4.74047046353335*sin(alp_i)/parallax_i, 0, -4.74047046353335*cos(alp_i)/parallax_i**2, 0, 0, 0], [-4.74047046353335*sin(dlt_i)*cos(alp_i)/parallax_i, -4.74047046353335*sin(alp_i)*cos(dlt_i)/parallax_i, 4.74047046353335*sin(alp_i)*sin(dlt_i)/parallax_i**2, 0, 0, 0], [cos(alp_i)*cos(dlt_i), -sin(alp_i)*sin(dlt_i), 0, 0, 0, 0]]
    vzHi = [[0, 0, 0, 0, 0, 0], [0, -rv_i*sin(dlt_i) - 4.74047046353335*pmdec_i*cos(dlt_i)/parallax_i, 4.74047046353335*pmdec_i*sin(dlt_i)/parallax_i**2, 0, -4.74047046353335*sin(dlt_i)/parallax_i, cos(dlt_i)], [0, 4.74047046353335*pmdec_i*sin(dlt_i)/parallax_i**2, 9.4809409270667*pmdec_i*cos(dlt_i)/parallax_i**3, 0, -4.74047046353335*cos(dlt_i)/parallax_i**2, 0], [0, 0, 0, 0, 0, 0], [0, -4.74047046353335*sin(dlt_i)/parallax_i, -4.74047046353335*cos(dlt_i)/parallax_i**2, 0, 0, 0], [0, cos(dlt_i), 0, 0, 0, 0]]
    return xHi,yHi,zHi,vxHi,vyHi,vzHi


In [27]:

def car_linear(i):
    rai,deci,plxi,pmrai,pmdeci,rvi = ra[i],dec[i],plx[i],pmra[i],pmdec[i],rv[i]
    alpha=deg2rad(rai)   #rad
    delta=deg2rad(deci)  #rad
    r=1000/plxi   #pc
    x=r*cos(delta)*cos(alpha)   #pc
    y=r*cos(delta)*sin(alpha)   #pc
    z=r*sin(delta)   #pc
    vx=rvi*cos(delta)*cos(alpha)+(-r*pmrai*sin(alpha)/1000-r*pmdeci*sin(delta)*cos(alpha)/1000)*4.740470463533348   #km/s
    vy=rvi*cos(delta)*sin(alpha)+(r*pmrai*cos(alpha)/1000-r*pmdeci*sin(delta)*sin(alpha)/1000)*4.740470463533348   #km/s
    vz=rvi*sin(delta)+r*pmdeci*cos(delta)/1000*4.740470463533348   #km/s

    cov_o = cov_obs(ra_e[i]/3600000,dec_e[i]/3600000,plx_e[i], pmra_e[i], pmdec_e[i], rv_e[i],ra_dec_corr[i], ra_plx_corr[i], ra_pmra_corr[i], ra_pmdec_corr[i],0,dec_plx_corr[i], dec_pmra_corr[i], dec_pmdec_corr[i],0,plx_pmra_corr[i], plx_pmdec_corr[i],0,pmra_pmdec_corr[i],0,0)
    J = J_car(rai,deci,plxi,pmrai,pmdeci,rvi)
    VC1 = J @ cov_o @ J.T          #linear var-cov matrix 
    sigx, sigy, sigz, sigvx, sigvy, sigvz = np.sqrt(np.diagonal(VC1))

    cor = array(corr(sigx, sigy, sigz, sigvx, sigvy, sigvz,VC1))
    xx,xy,xz,xvx,xvy,xvz = cor[0]
    yx,yy,yz,yvx,yvy,yvz = cor[1]
    zx,zy,zz,zvx,zvy,zvz = cor[2]
    vxx,vxy,vxz,vxvx,vxvy,vxvz = cor[3]
    vyx,vyy,vyz,vyvx,vyvy,vyvz = cor[4]
    vzx,vzy,vzz,vzvx,vzvy,vzvz = cor[5]
    return x,y,z,vx,vy,vz,sigx, sigy, sigz, sigvx, sigvy, sigvz,xy,xz,xvx,xvy,xvz,yz,yvx,yvy,yvz,zvx,zvy,zvz,vxvy,vxvz,vyvz

Second-order error propagation in Equatorial coordinate system

In [28]:
def car_2order(i):
    #Mean
    rai,deci,plxi,pmrai,pmdeci,rvi = ra[i],dec[i],plx[i],pmra[i],pmdec[i],rv[i]
    x,y,z,vx,vy,vz=ct(rai,deci,plxi,pmrai,pmdeci,rvi)
    cov_o = cov_obs(ra_e[i]/3600000,dec_e[i]/3600000,plx_e[i], pmra_e[i], pmdec_e[i], rv_e[i],ra_dec_corr[i], ra_plx_corr[i], ra_pmra_corr[i], ra_pmdec_corr[i],0,dec_plx_corr[i], dec_pmra_corr[i], dec_pmdec_corr[i],0,plx_pmra_corr[i], plx_pmdec_corr[i],0,pmra_pmdec_corr[i],0,0)
    
    J = J_car(rai,deci,plxi,pmrai,pmdeci,rvi)
    VC1 = J @ cov_o @ J.T          #linear var-cov matrix
    He = array(H(rai,deci,plxi,pmrai,pmdeci,rvi))
    He_cov = [np.dot(He[k], cov_o) for k in range(len(He))]
    b2 = array([sum(np.diagonal(He_cov[k])) for k in range(len(He_cov))])
    b1 = array([x,y,z,vx,vy,vz])
    bar = b1 + 0.5 * b2

    #Variance
    xbar,ybar,zbar,vxbar,vybar,vzbar = bar
    VC2 = np.diag([0.5 * sum(np.diagonal(h @ cov_o @ h @ cov_o)) for h in He])  #2-order VC 
    VC = VC1 +VC2  
    sigx, sigy, sigz, sigvx, sigvy, sigvz = np.sqrt(np.diagonal(VC))

    #Correlation coefficient
    cor = array(corr(sigx, sigy, sigz, sigvx, sigvy, sigvz,VC))
    xx,xy,xz,xvx,xvy,xvz = cor[0]
    yx,yy,yz,yvx,yvy,yvz = cor[1]
    zx,zy,zz,zvx,zvy,zvz = cor[2]
    vxx,vxy,vxz,vxvx,vxvy,vxvz = cor[3]
    vyx,vyy,vyz,vyvx,vyvy,vyvz = cor[4]
    vzx,vzy,vzz,vzvx,vzvy,vzvz = cor[5]
    return xbar,ybar,zbar,vxbar,vybar,vzbar,sigx, sigy, sigz, sigvx, sigvy, sigvz,xy,xz,xvx,xvy,xvz,yz,yvx,yvy,yvz,zvx,zvy,zvz,vxvy,vxvz,vyvz


Second-order error propagation Verification

In [40]:
## soerp was used to verify error propagation

from soerp import *   # uv, N, U, Exp, etc.
from soerp.umath import *  # sin, exp, sqrt, etc.
i = 0

#Gaussian distribution
ra1 = N(ra[i]*pi/180 , ra_e[i]/3600000*pi/180)  #ra unit:rad
dec1 = N(dec[i]*pi/180, dec_e[i]/3600000*pi/180)
plx1 = N(plx[i], plx_e[i])
pmra1 = N(pmra[i], pmra_e[i])  #pmra* 
pmdec1 = N(pmdec[i], pmdec_e[i])
rv1 = N(rv[i], rv_e[i])
dlt1 = dec1  
alpha=ra1
r=1000/plx1   #pc
x1=r*cos(dlt1)*cos(alpha)   #pc
y1=r*cos(dlt1)*sin(alpha)   #pc
z1=r*sin(dlt1)   #pc
vx1=rv1*cos(dlt1)*cos(alpha)+(-r*pmra1*sin(alpha)/1000-r*pmdec1*sin(dlt1)*cos(alpha)/1000)*4.740470463533348   #km/s
vy1=rv1*cos(dlt1)*sin(alpha)+(r*pmra1*cos(alpha)/1000-r*pmdec1*sin(dlt1)*sin(alpha)/1000)*4.740470463533348   #km/s
vz1=rv1*sin(dlt1)+r*pmdec1*cos(dlt1)/1000*4.740470463533348   #km/s

def gauss_linear(i):
    rai,deci,plxi,pmrai,pmdeci,rvi = ra[i],dec[i],plx[i],pmra[i],pmdec[i],rv[i]
    alpha=deg2rad(rai)   #rad
    delta=deg2rad(deci)  #rad
    r=1000/plxi   #pc
    x=r*cos(delta)*cos(alpha)   #pc
    y=r*cos(delta)*sin(alpha)   #pc
    z=r*sin(delta)   #pc
    vx=rvi*cos(delta)*cos(alpha)+(-r*pmrai*sin(alpha)/1000-r*pmdeci*sin(delta)*cos(alpha)/1000)*4.740470463533348   #km/s
    vy=rvi*cos(delta)*sin(alpha)+(r*pmrai*cos(alpha)/1000-r*pmdeci*sin(delta)*sin(alpha)/1000)*4.740470463533348   #km/s
    vz=rvi*sin(delta)+r*pmdeci*cos(delta)/1000*4.740470463533348   #km/s

    cov_o = cov_obs(ra_e[i]/3600000,dec_e[i]/3600000,plx_e[i], pmra_e[i], pmdec_e[i], rv_e[i],0,0,0,0,0,0,0,0,0,0,0,0,0,0,0)
    J = J_car(rai,deci,plxi,pmrai,pmdeci,rvi)
    VC1 = J @ cov_o @ J.T          #linear var-cov matrix 
    sigx, sigy, sigz, sigvx, sigvy, sigvz = np.sqrt(np.diagonal(VC1))

    cor = array(corr(sigx, sigy, sigz, sigvx, sigvy, sigvz,VC1))
    xx,xy,xz,xvx,xvy,xvz = cor[0]
    yx,yy,yz,yvx,yvy,yvz = cor[1]
    zx,zy,zz,zvx,zvy,zvz = cor[2]
    vxx,vxy,vxz,vxvx,vxvy,vxvz = cor[3]
    vyx,vyy,vyz,vyvx,vyvy,vyvz = cor[4]
    vzx,vzy,vzz,vzvx,vzvy,vzvz = cor[5]
    return x,y,z,vx,vy,vz,sigx, sigy, sigz, sigvx, sigvy, sigvz,xy,xz,xvx,xvy,xvz,yz,yvx,yvy,yvz,zvx,zvy,zvz,vxvy,vxvz,vyvz

def gauss_2order(i):

    rai,deci,plxi,pmrai,pmdeci,rvi = ra[i],dec[i],plx[i],pmra[i],pmdec[i],rv[i]
    x,y,z,vx,vy,vz=ct(rai,deci,plxi,pmrai,pmdeci,rvi)

    cov_o = cov_obs(ra_e[i]/3600000,dec_e[i]/3600000,plx_e[i], pmra_e[i], pmdec_e[i], rv_e[i],0,0,0,0,0,0,0,0,0,0,0,0,0,0,0)
    J = J_car(rai,deci,plxi,pmrai,pmdeci,rvi)
    VC1 = J @ cov_o @ J.T          #linear var-cov matrix
    He = array(H(rai,deci,plxi,pmrai,pmdeci,rvi))
    He_cov = [np.dot(He[k], cov_o) for k in range(len(He))]
    b2 = array([sum(np.diagonal(He_cov[k])) for k in range(len(He_cov))])
    b1 = array([x,y,z,vx,vy,vz])
    bar = b1 + 0.5 * b2
    xbar,ybar,zbar,vxbar,vybar,vzbar = bar
    VC2 = np.diag([0.5 * sum(np.diagonal(h @ cov_o @ h @ cov_o)) for h in He])
    VC = VC1 +VC2  #2-order Var ,1-order Cov
    sigx, sigy, sigz, sigvx, sigvy, sigvz = np.sqrt(np.diagonal(VC))

    cor = array(corr(sigx, sigy, sigz, sigvx, sigvy, sigvz,VC))
    xx,xy,xz,xvx,xvy,xvz = cor[0]
    yx,yy,yz,yvx,yvy,yvz = cor[1]
    zx,zy,zz,zvx,zvy,zvz = cor[2]
    vxx,vxy,vxz,vxvx,vxvy,vxvz = cor[3]
    vyx,vyy,vyz,vyvx,vyvy,vyvz = cor[4]
    vzx,vzy,vzz,vzvx,vzvy,vzvz = cor[5]
    return xbar,ybar,zbar,vxbar,vybar,vzbar,sigx, sigy, sigz, sigvx, sigvy, sigvz,xy,xz,xvx,xvy,xvz,yz,yvx,yvy,yvz,zvx,zvy,zvz,vxvy,vxvz,vyvz

di = gauss_2order(0)
ci = gauss_linear(0)
#'''
x1.describe()
print(di[0],di[6]**2)
print(ci[0],ci[6]**2)
y1.describe()
print(di[1],di[7]**2)
print(ci[1],ci[7]**2)
z1.describe()
print(di[2],di[8]**2)
print(ci[2],ci[8]**2)
'''
vx1.describe()
print(di[3],di[9]**2)
print(ci[3],ci[9]**2)
vy1.describe()
print(di[4],di[10]**2)
print(ci[4],ci[10]**2)
vz1.describe()
print(di[5],di[11]**2)
print(ci[5],ci[11]**2)
'''


SOERP Uncertain Value:
 > Mean...................  15.981422365703482
 > Variance...............  0.0008245877400311935
 > Skewness Coefficient...  0.01078082312295243
 > Kurtosis Coefficient...  3.0001549683630513

15.981422365703482 0.0008245877402245303
15.981370769227338 0.000824582415831829
SOERP Uncertain Value:
 > Mean................... -90.4016018290686
 > Variance...............  0.026385070942348
 > Skewness Coefficient... -0.010780823122952808
 > Kurtosis Coefficient...  3.0001549683630513

-90.4016018290686 0.026385070942403284
-90.40130996492876 0.026384900573051047
SOERP Uncertain Value:
 > Mean................... -63.33780956991267
 > Variance...............  0.012951858602066097
 > Skewness Coefficient... -0.010780823122952799
 > Kurtosis Coefficient...  3.00015496836305

-63.337809569912665 0.0129518586021729
-63.33760508199503 0.012951774971555979


'\nvx1.describe()\nprint(di[3],di[9]**2)\nprint(ci[3],ci[9]**2)\nvy1.describe()\nprint(di[4],di[10]**2)\nprint(ci[4],ci[10]**2)\nvz1.describe()\nprint(di[5],di[11]**2)\nprint(ci[5],ci[11]**2)\n'

In [43]:
##soerp verifies Hessian matrix

i = 0
rai,deci,plxi,pmrai,pmdeci,rvi = ra[i],dec[i],plx[i],pmra[i],pmdec[i],rv[i]
xHi,yHi,zHi,vxHi,vyHi,vzHi  = H(rai,deci,plxi,pmrai,pmdeci,rvi)
xh1 = np.array(x1.hessian([ra1,dec1,plx1,pmra1,pmdec1,rv1]))
print(xh1-xHi)
yh1 = np.array(y1.hessian([ra1,dec1,plx1,pmra1,pmdec1,rv1]))
zh1 = np.array(z1.hessian([ra1,dec1,plx1,pmra1,pmdec1,rv1]))
vxh1 = np.array(vx1.hessian([ra1,dec1,plx1,pmra1,pmdec1,rv1]))
#print(vxh1-vxHi)
vyh1 = np.array(vy1.hessian([ra1,dec1,plx1,pmra1,pmdec1,rv1]))
vzh1 = np.array(vz1.hessian([ra1,dec1,plx1,pmra1,pmdec1,rv1]))


[[-1.77635684e-15 -7.10542736e-15  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00]
 [-7.10542736e-15 -1.77635684e-15 -2.22044605e-16  0.00000000e+00
   0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00 -2.22044605e-16  1.11022302e-16  0.00000000e+00
   0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00]]


In [44]:
import random
# MC sample of linear propagation
def mc_car(i,N):
    cov_o = cov_obs(ras_e[i],dec_e[i],plx_e[i], pmra_e[i], pmdec_e[i], rv_e[i],ra_dec_corr[i], ra_plx_corr[i], ra_pmra_corr[i], ra_pmdec_corr[i],0,dec_plx_corr[i], dec_pmra_corr[i], dec_pmdec_corr[i],0,plx_pmra_corr[i], plx_pmdec_corr[i],0,pmra_pmdec_corr[i],0,0)
    mean_obs = (0,0,plx[i],pmra[i],pmdec[i],rv[i])
    mean_obs = array(mean_obs)
    a = np.random.multivariate_normal(mean_obs,cov_o,N)   #sample 
    
    #re-sample
    plxi = a[:,2]
    if np.min(plxi) <= 0:   # avoid plx<0
        a2 = np.random.multivariate_normal(mean_obs,cov_o,2*N)   #sample2
        plx2i = a2[:,2]
        a2p = a2[np.where(plx2i>0)]
        sample_list = [i for i in range(len(a2p))]
        sample_list = random.sample(sample_list, N) 
        a = a2p[sample_list,:]
    
    a[:,0] = a[:,0]/3600000 / np.cos(np.deg2rad(dec[i]))  + ra[i]  #ra(deg)
    a[:,1] = a[:,1]/3600000 + dec[i]   #dec(deg)
    CT_non_linear_data = []
    CT_non_linear_data_i = ct(a[:,0],a[:,1],a[:,2],a[:,3],a[:,4],a[:,5])
    cov_car = np.cov(CT_non_linear_data_i, bias=True)
    x,y,z,vx,vy,vz = np.mean(CT_non_linear_data_i,axis=1)
    sigx, sigy, sigz, sigvx, sigvy, sigvz = np.sqrt(cov_car[[0,1,2,3,4,5],[0,1,2,3,4,5]])
    cor = array(corr(sigx, sigy, sigz, sigvx, sigvy, sigvz,cov_car))
    xx,xy,xz,xvx,xvy,xvz = cor[0]
    yx,yy,yz,yvx,yvy,yvz = cor[1]
    zx,zy,zz,zvx,zvy,zvz = cor[2]
    vxx,vxy,vxz,vxvx,vxvy,vxvz = cor[3]
    vyx,vyy,vyz,vyvx,vyvy,vyvz = cor[4]
    return x,y,z,vx,vy,vz,sigx, sigy, sigz, sigvx, sigvy, sigvz,xy,xz,xvx,xvy,xvz,yz,yvx,yvy,yvz,zvx,zvy,zvz,vxvy,vxvz,vyvz

In [45]:
i = 0
print(mc_car(i,10000))
print(car_2order(i))
print(car_linear(i))

(15.981595615652568, -90.40258184669953, -63.33849619676671, 12.349161036996243, 32.930175393952965, -15.549383562205954, 0.028808967781037028, 0.16296276918446786, 0.11417612577500179, 0.4036058470378937, 2.2787430159934, 1.5967156098289828, -0.9999999999999625, -0.9999999999999605, 0.06552315584366387, 0.021232326890574586, -0.023552996803110793, 0.9999999999999981, -0.06552315731913945, -0.02123232625623402, 0.02355299727886558, -0.06552315705673063, -0.02123232671491473, 0.023552996703047783, -0.9959469577486866, -0.9988037718696205, 0.9989799830073467)
(15.981422365715206, -90.40160182916576, -63.337809569771025, 12.349091064076678, 32.929851552067326, -15.549373019419095, 0.028715642339448646, 0.16243487467302653, 0.11380615387487512, 0.40661920269502144, 2.2941336134504997, 1.6077760386061801, -0.9999935428403139, -0.9999935428344582, 0.06995069728844734, 0.016057551762945578, -0.028239055517744662, 0.9999935429555991, -0.06995074168083767, -0.01605754972890473, 0.02823904854345

Transform to Galactic coordinate system 

In [46]:
# Coordinates of North Galactic pole (NGP)

c = SkyCoord(l=0*u.deg, b=90*u.deg, frame='galactic')
cg = c.transform_to('icrs')
a = cg.ra.deg  #ra of NGP
d = cg.dec.deg  #dec of NGP
print(a,d)

c = SkyCoord(ra=0*u.deg, dec=90*u.deg, frame='icrs')
cg = c.transform_to('galactic')
l = cg.l.deg  #l of NGP
d = cg.b.deg
print(l,d)

192.85947789477606 27.128252414968028
122.93192525541662 27.128252414968


In [47]:
def concat(a, b): # combine a and b
    lena = len(a)
    lenb = len(b)
    left = np.row_stack((a, np.zeros((lenb, lena))))
    right = np.row_stack((np.zeros((lena, lenb)), b))
    result = np.hstack((left, right)) 
    return result


rad2deg = 180/pi
alphaG = 192.85947789477606/rad2deg   #ra of NGP
deltaG = 27.128252414968028/rad2deg    #dec of NGP
lN = 122.93192525541662/rad2deg     #l of NGP

degcel = np.matrix([[-sin(alphaG),cos(alphaG),0],[-sin(deltaG)*cos(alphaG),-sin(deltaG)*sin(alphaG),cos(deltaG)]])
deggal = np.matrix([[sin(lN),cos(lN)],[-cos(lN),sin(lN)]])
galc = np.dot(deggal,degcel)
zc = np.matrix([cos(deltaG)*cos(alphaG),cos(deltaG)*sin(alphaG),sin(deltaG)])
gal = np.row_stack((galc,zc))
tran = np.matrix(concat(gal,gal))

i = 0
rai,deci,plxi,pmrai,pmdeci,rvi = ra[i],dec[i],plx[i],pmra[i],pmdec[i],rv[i]
xb,yb,zb,vxb,vyb,vzb = ct(rai,deci,plxi,pmrai,pmdeci,rvi)
    
l = np.matrix([[xb],[yb],[zb],[vxb],[vyb],[vzb]])
galc = array(np.dot(tran,l))
x, y, z, vx, vy, vz = galc[:,0]

In [48]:
# verification
c = SkyCoord(ra=rai*u.deg, dec=deci*u.deg, distance=1/plxi*u.kpc, pm_ra_cosdec=pmrai*u.mas/u.yr, pm_dec=pmdeci*u.mas/u.yr, radial_velocity=rvi*u.km/u.s, frame='icrs')
cg = c.transform_to('galactic')
x2 = cg.cartesian.x.to(u.pc)
y2 = cg.cartesian.y.to(u.pc)
z2 = cg.cartesian.z.to(u.pc)
vx2 = cg.velocity.d_x.to(u.km/u.s)
vy2 = cg.velocity.d_y.to(u.km/u.s)
vz2 = cg.velocity.d_z.to(u.km/u.s)
print(x,y,z,vx,vy,vz)
print(x2,y2,z2,vx2,vy2,vz2)

108.72782025413662 0.7976730536374674 -24.841056600818355 -21.91646376798148 -20.16141373846515 -24.327680284104595
108.72782025413659 pc 0.797673053637496 pc -24.84105660081832 pc -21.916463767981462 km / s -20.161413738465168 km / s -24.327680284104577 km / s


In [49]:
def diff2(i):  #Second-order mean transformation and covariance atrix in Equatorial coordinate system
    rai, deci, plxi, pmrai, pmdeci, rvi = ra[i], dec[i], plx[i], pmra[i], pmdec[i], rv[i]
    x, y, z, vx, vy, vz = ct(rai, deci, plxi, pmrai, pmdeci, rvi)
    ra_ei = ra_e[i]/3600000  # mas -> deg
    dec_ei = dec_e[i]/3600000  # mas -> deg

    cov_o = cov_obs(ra_ei, dec_ei, plx_e[i], pmra_e[i], pmdec_e[i], rv_e[i], ra_dec_corr[i], ra_plx_corr[i], ra_pmra_corr[i], ra_pmdec_corr[i],
                    0, dec_plx_corr[i], dec_pmra_corr[i], dec_pmdec_corr[i], 0, plx_pmra_corr[i], plx_pmdec_corr[i], 0, pmra_pmdec_corr[i], 0, 0)
    delta = deg2rad(deci)  # rad
    J = J_car(rai, deci, plxi, pmrai, pmdeci, rvi)
    VC1 = J @ cov_o @ J.T  # linear var-cov matrix
    He = array(H(rai, deci, plxi, pmrai, pmdeci, rvi))
    He_cov = [np.dot(He[k], cov_o) for k in range(len(He))]
    b2 = array([sum(np.diagonal(He_cov[k])) for k in range(len(He_cov))])
    b1 = array([x, y, z, vx, vy, vz])
    bar = b1 + 0.5 * b2
    xbar, ybar, zbar, vxbar, vybar, vzbar = bar
    VC2 = np.diag([0.5 * sum(np.diagonal(h @ cov_o @ h @ cov_o)) for h in He])
    VC = VC1 + VC2  
    return xbar, ybar, zbar, vxbar, vybar, vzbar, VC


def gal_2order(i):# in galactic coordinate system
    xb,yb,zb,vxb,vyb,vzb,VC = diff2(i)
    l = np.matrix([[xb],[yb],[zb],[vxb],[vyb],[vzb]])
    #print(ct)
    VCi = tran @ VC @ tran.T
    galc = array(np.dot(tran,l))
    x, y, z, vx, vy, vz = galc[:,0]
    sigx, sigy, sigz, sigvx, sigvy, sigvz = np.sqrt(diagonal(VCi))
    cor = array(corr(sigx, sigy, sigz, sigvx, sigvy, sigvz,VCi))
    xx,xy,xz,xvx,xvy,xvz = cor[0]
    yx,yy,yz,yvx,yvy,yvz = cor[1]
    zx,zy,zz,zvx,zvy,zvz = cor[2]
    vxx,vxy,vxz,vxvx,vxvy,vxvz = cor[3]
    vyx,vyy,vyz,vyvx,vyvy,vyvz = cor[4]
    vzx,vzy,vzz,vzvx,vzvy,vzvz = cor[5]
    return x,y,z,vx,vy,vz,sigx, sigy, sigz, sigvx, sigvy, sigvz,xy,xz,xvx,xvy,xvz,yz,yvx,yvy,yvz,zvx,zvy,zvz,vxvy,vxvz,vyvz

def mc_gal(i,N):
    cov_o = cov_obs(ras_e[i],dec_e[i],plx_e[i], pmra_e[i], pmdec_e[i], rv_e[i],ra_dec_corr[i], ra_plx_corr[i], ra_pmra_corr[i], ra_pmdec_corr[i],0,dec_plx_corr[i], dec_pmra_corr[i], dec_pmdec_corr[i],0,plx_pmra_corr[i], plx_pmdec_corr[i],0,pmra_pmdec_corr[i],0,0)
    mean_obs = (0,0,plx[i],pmra[i],pmdec[i],rv[i])
    mean_obs = array(mean_obs)
    a = np.random.multivariate_normal(mean_obs,cov_o,N)   #sample 
    
    #re-sample
    plxi = a[:,2]
    if np.min(plxi) <= 0:
        a2 = np.random.multivariate_normal(mean_obs,cov_o,2*N)   #sample2
        plx2i = a2[:,2]
        a2p = a2[np.where(plx2i>0)]
        sample_list = [i for i in range(len(a2p))]
        sample_list = random.sample(sample_list, N)  #随机行数
        a = a2p[sample_list,:]
    
    a[:,0] = a[:,0]/3600000 / np.cos(np.deg2rad(dec[i]))  + ra[i]  #ra(deg) 
    a[:,1] = a[:,1]/3600000 + dec[i]   #dec(deg)

    gal_non_linear_data_i = []
    for k in range(len(a[:,0])):
      xb,yb,zb,vxb,vyb,vzb = ct(a[k,0],a[k,1],a[k,2],a[k,3],a[k,4],a[k,5])
      l = np.mat([[xb],[yb],[zb],[vxb],[vyb],[vzb]])
      galc = array(np.dot(tran,l))
      x, y, z, vx, vy, vz = galc[:,0]
      gal_non_linear_data_i.append([x, y, z, vx, vy, vz])
    gal_non_linear_data_i = np.array(gal_non_linear_data_i)
    gal_non_linear_data_i = np.array([gal_non_linear_data_i[:,0],gal_non_linear_data_i[:,1],gal_non_linear_data_i[:,2],gal_non_linear_data_i[:,3],gal_non_linear_data_i[:,4],gal_non_linear_data_i[:,5]])
    cov_car = np.cov(gal_non_linear_data_i, bias=True)
    x,y,z,vx,vy,vz = np.mean(gal_non_linear_data_i,axis=1)
    sigx, sigy, sigz, sigvx, sigvy, sigvz = np.sqrt(cov_car[[0,1,2,3,4,5],[0,1,2,3,4,5]])
    cor = array(corr(sigx, sigy, sigz, sigvx, sigvy, sigvz,cov_car))
    xx,xy,xz,xvx,xvy,xvz = cor[0]
    yx,yy,yz,yvx,yvy,yvz = cor[1]
    zx,zy,zz,zvx,zvy,zvz = cor[2]
    vxx,vxy,vxz,vxvx,vxvy,vxvz = cor[3]
    vyx,vyy,vyz,vyvx,vyvy,vyvz = cor[4]
    return x,y,z,vx,vy,vz,sigx, sigy, sigz, sigvx, sigvy, sigvz,xy,xz,xvx,xvy,xvz,yz,yvx,yvy,yvz,zvx,zvy,zvz,vxvy,vxvz,vyvz

The final function (2od mean + MC4 error)

In [50]:
import random
def tot_2d_mc4(i):
    xcar, ycar, zcar, vxcar, vycar, vzcar, VC = diff2(i)
    l = np.matrix([[xcar], [ycar], [zcar], [vxcar], [vycar], [vzcar]])
    VCi = tran @ VC @ tran.T
    galc = array(np.dot(tran, l))
    xgal,ygal,zgal,vxgal,vygal,vzgal = galc[:, 0]

    cov_o = cov_obs(ras_e[i],dec_e[i],plx_e[i], pmra_e[i], pmdec_e[i], rv_e[i],ra_dec_corr[i], ra_plx_corr[i], ra_pmra_corr[i], ra_pmdec_corr[i],0,dec_plx_corr[i], dec_pmra_corr[i], dec_pmdec_corr[i],0,plx_pmra_corr[i], plx_pmdec_corr[i],0,pmra_pmdec_corr[i],0,0)
    mean_obs = (0,0,plx[i],pmra[i],pmdec[i],rv[i])
    mean_obs = array(mean_obs)
    a = np.random.multivariate_normal(mean_obs,cov_o,10**4)   #sample 
    
    #re-sample
    plxi = a[:,2]
    if np.min(plxi) <= 0:
        a2 = np.random.multivariate_normal(mean_obs,cov_o,2*10**4)   #sample2
        plx2i = a2[:,2]
        a2p = a2[np.where(plx2i>0)]
        sample_list = [i for i in range(len(a2p))]
        sample_list = random.sample(sample_list, 10**4) 
        a = a2p[sample_list,:]

    a[:,0] = a[:,0]/3600000 / np.cos(np.deg2rad(dec[i]))  + ra[i]  #ra(deg) 
    a[:,1] = a[:,1]/3600000 + dec[i]   #dec(deg)
    pm_ra = a[:,3]
    pm_dec = a[:,4]
    pmi = np.sqrt(pm_ra**2+pm_dec**2)
    plx_0 = a[:,2]
    ds = 1000/plx_0   #distance
    d = np.mean(ds)
    d_e = np.sqrt(np.var(ds))
    vts = pmi * ds*4.740470463533348   #km/s  #transverse velocity
    vt = np.mean(vts)
    vt_e = np.sqrt(np.var(vts))

    #equatorial 
    CT_non_linear_data = []
    CT_non_linear_data_i = ct(a[:,0],a[:,1],a[:,2],a[:,3],a[:,4],a[:,5])
    cov_car = np.cov(CT_non_linear_data_i, bias=True)
    sigx_c, sigy_c, sigz_c, sigvx_c, sigvy_c, sigvz_c = np.sqrt(cov_car[[0,1,2,3,4,5],[0,1,2,3,4,5]])
    cor = array(corr(sigx_c, sigy_c, sigz_c, sigvx_c, sigvy_c, sigvz_c,cov_car))
    cxx,cxy,cxz,cxvx,cxvy,cxvz = cor[0]
    cyx,cyy,cyz,cyvx,cyvy,cyvz = cor[1]
    czx,czy,czz,czvx,czvy,czvz = cor[2]
    cvxx,cvxy,cvxz,cvxvx,cvxvy,cvxvz = cor[3]
    cvyx,cvyy,cvyz,cvyvx,cvyvy,cvyvz = cor[4]

    # galactic
    VCi = tran @ cov_car @ tran.T
    sigx_g, sigy_g, sigz_g, sigvx_g, sigvy_g, sigvz_g = np.sqrt(diagonal(VCi))
    cor = array(corr(sigx_g, sigy_g, sigz_g, sigvx_g, sigvy_g, sigvz_g, VCi))
    gxx,gxy,gxz,gxvx,gxvy,gxvz = cor[0]
    gyx,gyy,gyz,gyvx,gyvy,gyvz = cor[1]
    gzx,gzy,gzz,gzvx,gzvy,gzvz = cor[2]
    gvxx,gvxy,gvxz,gvxvx,gvxvy,gvxvz = cor[3]
    gvyx,gvyy,gvyz,gvyvx,gvyvy,gvyvz = cor[4]
    
    return d,d_e,vt,vt_e, xcar,sigx_c,xgal,sigx_g, ycar,sigy_c,ygal,sigy_g, zcar,sigz_c,zgal,sigz_g, vxcar,sigvx_c,vxgal,sigvx_g, vycar,sigvy_c,vygal,sigvy_g, vzcar,sigvz_c,vzgal,sigvz_g, \
        cxy,gxy,cxz,gxz,cxvx,gxvx,cxvy,gxvy,cxvz,gxvz, cyz,gyz,cyvx,gyvx,cyvy,gyvy,cyvz,gyvz, czvx,gzvx,czvy,gzvy,czvz,gzvz, cvxvy,gvxvy,cvxvz,gvxvz, cvyvz,gvyvz


In [51]:
i = 7
tot_2d_mc4(i)

(137.67963515046281,
 0.4029913787806335,
 29729.64566694086,
 93.83472720135306,
 111.90227216907483,
 0.32753063022259105,
 73.93610324530557,
 0.21640614161939245,
 -70.99774478748957,
 0.2078057582617748,
 58.98410610874779,
 0.17264262033233296,
 -37.33659280299599,
 0.10928176641488568,
 -100.05572101134162,
 0.2928565500133297,
 -4.32071978210477,
 0.5940893659751728,
 -35.19041668429978,
 0.39741629394401484,
 37.88327096269352,
 0.38365980592910004,
 -15.375589407564703,
 0.31231730877651653,
 4.834016059564445,
 0.1978862954818357,
 -1.5506042331286363,
 0.5327421471371119,
 -0.999999999999998,
 0.9999999999999952,
 -0.9999999999999932,
 -0.9999999999999976,
 0.08407927375057753,
 -0.1764776936421837,
 0.20447427364074788,
 -0.03827308802897269,
 -0.011758826107987575,
 -0.11251274769256128,
 0.9999999999999952,
 -0.9999999999999978,
 -0.08407927313773125,
 -0.17647769149193407,
 -0.2044742746962634,
 -0.038273088514343544,
 0.011758824643916518,
 -0.11251274893092689,
 -0.08

Transform data

In [ ]:
name = 'Gaia_rv_dr3_%s' %(id)

g2 = []
for i in range(len(data)):
    gk = tot_2d_mc4(i)   
    g2.append(gk)

g2 = np.array(g2)
dataframe = pd.DataFrame({'DR3 designation':designation, 'r_eqt': g2[:,0],'r_eqt_error': g2[:,1],'vt_eqt': g2[:,2],'vt_eqt_error': g2[:,3], 'x_eqt': g2[:,4],'x_eqt_error': g2[:,5],'x_gal': g2[:,6], 'x_gal_error': g2[:,7], 'y_eqt': g2[:,8], 'y_eqt_error': g2[:,9], 'y_gal': g2[:,10], 'y_gal_error': g2[:,11], 'z_eqt': g2[:,12],'z_eqt_error': g2[:,13],'z_gal': g2[:,14],'z_gal_error': g2[:,15],\
    'vx_eqt': g2[:,16], 'vx_eqt_error': g2[:,17],'vx_gal': g2[:,18], 'vx_gal_error': g2[:,19], 'vy_eqt': g2[:,20], 'vy_eqt_error': g2[:,21],'vy_gal': g2[:,22], 'vy_gal_error': g2[:,23], 'vz_eqt': g2[:,24],'vz_eqt_error': g2[:,25],'vz_gal': g2[:,26],'vz_gal_error': g2[:,27],\
        'x_y_eqt_corr': g2[:,28],'x_y_gal_corr': g2[:,29],'x_z_eqt_corr': g2[:,30],'x_z_gal_corr': g2[:,31],'x_vx_eqt_corr': g2[:,32],'x_vx_gal_corr': g2[:,33],'x_vy_eqt_corr': g2[:,34],'x_vy_gal_corr': g2[:,35],'x_vz_eqt_corr': g2[:,36],'x_vz_gal_corr': g2[:,37],\
            'y_z_eqt_corr': g2[:,38],'y_z_gal_corr': g2[:,39],'y_vx_eqt_corr': g2[:,40],'y_vx_gal_corr': g2[:,41],'y_vy_eqt_corr': g2[:,42],'y_vy_gal_corr': g2[:,43],'y_vz_eqt_corr': g2[:,44],'y_vz_gal_corr': g2[:,45],\
                'z_vx_eqt_corr': g2[:,46],'z_vx_gal_corr': g2[:,47],'z_vy_eqt_corr': g2[:,48],'z_vy_gal_corr': g2[:,49],'z_vz_eqt_corr': g2[:,50],'z_vz_gal_corr': g2[:,51],'vx_vy_eqt_corr': g2[:,52],'vx_vy_gal_corr': g2[:,53],'vx_vz_eqt_corr': g2[:,54],'vx_vz_gal_corr': g2[:,55],'vy_vz_eqt_corr': g2[:,56],'vy_vz_gal_corr': g2[:,57]})
dataframe.to_csv(r"GDR3-2od+mc4/%s.csv" %(name), index=False, sep=',')